
<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<font color=0F5298 size=7>
Digital Image Processing <br>
<font color=2565AE size=5>
Department Of Mathematical Sciences <br>
Spring 2025<br>
<font color=3C99D size=5>
Assignment 1 <br>
</div>


---



### Full Name : Kasra Siavashpour
### Student Number :99109903
___

## Introduction

In this assignment, we are going to investigate some of the basic methods of image enhancement.

## Section 1: Image Enhancement

Importing necessary Libraries. You cannot import and use any other libraries

In [2]:
# Do not modify this cell
import cv2
import numpy as np
import matplotlib.pylab as plt


### Q1: Intensity Enhancement


In this question, you will implement some of the intensity processing functions and apply them on a given image. In the final part, you will be asked to use these functions to enhance the intensity of a given image.
x
<ol>
<li>
 <b>Negative Transform :</b> $y=255-x$

 <li>
 <b>Log Transform :</b> $y=\frac{255 \log(1+ \alpha x)}{\log(1+255 \alpha)}$

 <li>
 <b>Gamma Transform :</b> $y=255(\frac{x}{255})^{\gamma}$

 </ol>

In the following cells, complete the functions for

In [3]:
def negative_transform(img: np.ndarray):
    # Negative transform: y = 255 - x
    return 255 - img

In [4]:
def log_transform(img: np.ndarray, alpha):
    # Convert image to float for accurate computation
    img_float = img.astype(np.float32)
    # Log transform: y = 255 * log(1 + alpha*x) / log(1 + 255*alpha)
    transformed = 255 * (np.log(1 + alpha * img_float) / np.log(1 + 255 * alpha))
    # Clip values to [0, 255] and convert back to uint8
    return np.uint8(np.clip(transformed, 0, 255))


In [5]:
def gamma_transform(img: np.ndarray, gamma):
    # Convert image to float for accurate computation
    img_float = img.astype(np.float32)
    # Gamma transform: y = 255 * (x/255)^gamma
    transformed = 255 * ((img_float / 255) ** gamma)
    # Clip values to [0, 255] and convert back to uint8
    return np.uint8(np.clip(transformed, 0, 255))

Now, apply these transformations on `Q1-input1.jpg` (on RGB channels) and save the output images in `Q1-output-neg.jpg`, `Q1-output-log.jpg` and `Q1-output-gamma.jpg`, respectively. You can set $\alpha$ and $\gamma$ whatever you want, but effect on the intensity of the image should be noticable.

In [6]:
# Read the first input image
img1 = cv2.imread('input-images/Q1-input1.jpg')

# Apply the transformations on each channel (functions work element-wise)
img1_neg = negative_transform(img1)
img1_log = log_transform(img1, alpha=0.05)   # alpha chosen to make the log effect visible
img1_gamma = gamma_transform(img1, gamma=0.5)  # gamma < 1 brightens the image

# Save the output images
cv2.imwrite('output-images/Q1-output-neg.jpg', img1_neg)
cv2.imwrite('output-images/Q1-output-log.jpg', img1_log)
cv2.imwrite('output-images/Q1-output-gamma.jpg', img1_gamma)


True

Finally, use the functions you implemented to enhance the intensity of `Q1-input2.jpg`. You may use any of the functions in anyway you desire. Your final score on the question depends on the quality of the output. Save the output image as `Q1-output-enhanced.jpg`.

In [7]:
img2 = cv2.imread('input-images/Q1-input2.jpg')

# Enhance the intensity of the second image by combining transformations.
# Here, we first apply a gamma transform (with gamma=0.7 to brighten the image) 
# and then a log transform (with alpha=0.05) to boost details in darker regions.
img2_enhanced = log_transform(gamma_transform(img2, gamma=0.7), alpha=0.05)

# Save the enhanced image
cv2.imwrite('output-images/Q1-output-enhanced.jpg', img2_enhanced)

True

### Q2: Histogram Specification


In this question, we will implement histogram specification for intensity enhancement.

Complete the following code to specify the histogram of the input image, with a given histogram `hist1`, to target histogram `hist2`.

**Note**: Assume that the input image has only one channel, i.e, it is a $H \times W$ numpy array.

In [8]:
def histogram_specification(img: np.ndarray, hist1: np.ndarray, hist2: np.ndarray):
    # Compute the cumulative distribution function (CDF) for both histograms
    cdf1 = np.cumsum(hist1)
    cdf2 = np.cumsum(hist2)
    
    # Normalize the CDFs to be in the range [0,1]
    cdf1_normalized = cdf1 / cdf1[-1]
    cdf2_normalized = cdf2 / cdf2[-1]
    
    # Build the lookup table (mapping) from source intensities to target intensities.
    # For each intensity value i in the source, find the intensity j in the target
    # such that the difference between cdf1[i] and cdf2[j] is minimized.
    mapping = np.zeros(256, dtype=np.uint8)
    for i in range(256):
        # Find the target intensity that minimizes the absolute difference
        diff = np.abs(cdf1_normalized[i] - cdf2_normalized)
        mapping[i] = np.argmin(diff)
    
    # Apply the mapping to the image.
    # Since img contains intensity values in [0,255], we can use the mapping as a LUT.
    return mapping[img]

To test the above code, open `Q2-input1.jpg` and `Q2-input2.jpg`, and convert the histogram of the first one to the second one. Save the resulting image as `Q2-output.jpg`. The aim of specifying the histograms is to enhance the first image, so your score depends on the quality of your result.

In [9]:
img1 = cv2.imread('input-images/Q2-input1.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('input-images/Q2-input2.jpg', cv2.IMREAD_GRAYSCALE)

# Compute histograms for both images.
# hist1: histogram of the source image; hist2: histogram of the target image.
hist1 = cv2.calcHist([img1], [0], None, [256], [0, 256]).flatten()
hist2 = cv2.calcHist([img2], [0], None, [256], [0, 256]).flatten()

# Apply histogram specification: adjust the histogram of img1 to match that of img2.
output_img = histogram_specification(img1, hist1, hist2)

# Save the resulting image
cv2.imwrite('output-images/Q2-output.jpg', output_img)

True

### Q3: Color Processing

In image `Q3-input.jpg`, change the color of the flowers red, blue and yellow to violet, pink and orange respectively. Save the result as `Q3-output.jpg`. Your score of this question depends on the quality of the output image.

In [22]:

img = cv2.imread('input-images/Q3-input.jpg')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define HSV ranges for red (split into two parts)
lower_red1 = np.array([0, 70, 50])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 70, 50])
upper_red2 = np.array([180, 255, 255])
mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
mask_red = cv2.bitwise_or(mask_red1, mask_red2)

# Optionally refine the mask using morphological operations
kernel = np.ones((3,3), np.uint8)
mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_OPEN, kernel, iterations=2)

# --- Contour filtering for the red mask ---
# Find contours in the red mask
contours, _ = cv2.findContours(mask_red.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create an empty refined mask
refined_mask_red = np.zeros_like(mask_red)

# Filter contours by area (adjust the threshold as needed)
for cnt in contours:
    if cv2.contourArea(cnt) > 100:  # only keep regions with significant area
        cv2.drawContours(refined_mask_red, [cnt], -1, 255, thickness=cv2.FILLED)

# Replace red regions with the target color (violet) using the refined mask
output = img.copy()
violet = (238, 130, 238)  # BGR for violet
output[refined_mask_red > 0] = violet

# You would similarly refine masks for blue and yellow,
# then replace them with pink and orange respectively.

# For demonstration, here's a similar approach for blue:
lower_blue = np.array([90, 50, 50])
upper_blue = np.array([130, 255, 255])
mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
mask_blue = cv2.morphologyEx(mask_blue, cv2.MORPH_OPEN, kernel, iterations=2)
contours, _ = cv2.findContours(mask_blue.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
refined_mask_blue = np.zeros_like(mask_blue)
for cnt in contours:
    if cv2.contourArea(cnt) > 100:
        cv2.drawContours(refined_mask_blue, [cnt], -1, 255, thickness=cv2.FILLED)
pink = (193, 182, 255)  # BGR for pink
output[refined_mask_blue > 0] = pink

# And for yellow:
lower_yellow = np.array([20, 70, 50])
upper_yellow = np.array([35, 255, 255])
mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_OPEN, kernel, iterations=2)
contours, _ = cv2.findContours(mask_yellow.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
refined_mask_yellow = np.zeros_like(mask_yellow)
for cnt in contours:
    if cv2.contourArea(cnt) > 100:
        cv2.drawContours(refined_mask_yellow, [cnt], -1, 255, thickness=cv2.FILLED)
orange = (0, 165, 255)  # BGR for orange
output[refined_mask_yellow > 0] = orange

# Save the result
cv2.imwrite('output-images/Q3-output.jpg', output)


True

## Section 2: Spatial Filters

### Q4: Blurring

In this question, you are going to implement a function to blur a given image using a specified method. Your function should support box filter, gaussian filter and median filter. It is permitted to use `opencv` to perform the bluring.

In [ ]:
def blur(img: np.ndarray, kernel_size: int, filter: string):
  # TODO: implement blurring for box filter, Gaussian Blurring and median blurring
  pass

Test your code with kernel size $k=3, 11, 25$ for each of filters on `Q4-input.jpg`. Save the output images with the following format:
<ul>
<li> Box filter with kernel size $k=3, 11, 25$ as Q4-output-box3.jpg, Q4-output-box11.jpg and Q4-output-box25.jpg.

<li> Gaussian filter with kernel size $k=3, 11, 25$ as Q4-output-gauss3.jpg, Q4-output-gauss11.jpg and Q4-output-gauss25.jpg (you can set $\sigma$ whaever you wish, but blurring should be observable).

<li> Median filter with kernel size $k=3, 11, 25$ as Q4-output-med3.jpg, Q4-output-med11.jpg and Q4-output-med25.jpg.
</ul>

In [ ]:
img=cv2.imread('Q4-input.jpg')
# TODO: Apply the filters

### Q5: Sharpening

In this question, you are going to sharpen a given image using Gaussian Filter.


Let $f$ be the source image and $g$ be the Gaussian kernel, then $f*g$ is the convolution of the image with this kernel, which results in a blurred image.

To sharpen the image, we can generate an unsharp mask by subtracting the blurred image from the original image:
$$m=f-f*g$$
This mask is a transparent image which has higher intensity on the edges of the original image. Now we may add this mask with a weight $\alpha$ to sharpen the original image:
$$f'=f+ \alpha m$$

First, complete the following function to generate a Gaussian kernel with a given size and $\sigma$. A $(2k+1) \times (2k+1)$ Gaussian kernel has the value $e^{\frac{-(x^2+y^2)}{2 \sigma^2}}$ at point $(x,y)$ (Center of the kernel has coordinates $(0,0)$, and we have $x,y \in [-k,k]$). You should also normalize the kernel (sum of the entries should app up to 1)

In [ ]:
def gaussian_kernel(kernel_size: int, sigma):
  # TODO: create the gaussian kernel with given size and sigma
  pass

Next, complete the following function to sharpen a given image and Gaussian kernel and $\alpha$. Your function should return both the sharpened image and the unsharp mask.

In [ ]:
def gaussian_sharpen(img: np.ndarray, kernel: np.ndarray, alpha: float):
  # TODO: sharpen the image and return both the result and unsharp mask
  pass

Now to test your code, open `Q5-input.jpg` and sharpen it using the above function. Use a kernel size of 7 and choose a suitable value of $\sigma$ and $\alpha$ to get a desirable result. Save the sharpened image as `Q5-output-1.jpg` and the unsharp mask (grayscale image) as `Q5-output-2.jpg`. Your score on this part depends on the quality of the result.

In [ ]:
img=cv2.imread('Q5-input.jpg')
# TODO: Sharpen the image and save the result and the unsharp mask

### Q6: Noise removal

## A) ACE Function
Implement the ACE function using ready-made functions for histogram equalization and padding. Apply the function with different grid sizes on ACE.jpg image and report the output for each one. Also specify which output is the best.

## B) CLAHE Function
Implement the CLAHE function and apply it to CLAHE.jpg image. Also specify which output is the best. In this function, test different values for the clipping threshold and grid size, analyze the effect of each, and output the best result.

## C) add_noise Function
Complete the add_noise function to add Gaussian or Salt-and-Pepper noise to the image. Using existing library functions is not allowed. Read the noise.tif image in grayscale and add Gaussian and Salt-and-Pepper noise separately to the image and display the output.

## D) denoise Function
Save the image resulting from adding Salt-and-Pepper noise and denoise it using an appropriate filter.

In [ ]:
def ACE(image, gridSize):
  '''
  inputs:
    image (ndarray): input image
    gridSize (tuple): window size for calculating histogram equalization
  output:
    output (ndarray): improved image
  '''

  ###############################################
  ############# YOUR CODE GOES HERE #############

  output = ...

  ###############################################
  return output

In [ ]:
###############################################
######## PLOT INPUT AND OUTPUT IMAGES##########


###############################################

In [ ]:
def CLAHE(image, gridSize, threshold):
  '''
  inputs:
    image (ndarray): input image
    gridSize (tuple): window size for calculating histogram equalization
    threshold (int): threshold for contrast limiting
  output:
    output (ndarray): improved image
  '''
  ###############################################
  ############# YOUR CODE GOES HERE #############

  output = ...

  ###############################################
  return output

In [ ]:
###############################################
######## PLOT INPUT AND OUTPUT IMAGES##########


###############################################

In [ ]:
def add_noise(image, noise_type, args):

  '''
  inputs:
    image (ndarray): This is the input image represented as a NumPy array.

    noise_type (int): This parameter specifies which type of noise to add.
    It can take the following values:
    0: Gaussian Noise
    1: Salt-and-Pepper Noise

    args (list): A list of arguments that contains additional parameters needed for each noise mode.The required arguments are different depending on the noise_type.

  output:
    output_image (ndarray): noisy image.


  1. Gaussian Noise (noise_type = 0):

    Arguments (passed via args):
    mean: The mean of the Gaussian distribution (center value of noise).
    sigma: the standard deviation (sigma) of the Gaussian distribution.

    How it works:
    Generate Gaussian noise with this mean and sigma that has the same shape as the input image.
    Add this noise to the original image to create the noisy image.

  2. Salt-and-Pepper Noise (noise_type = 1):

    Arguments (passed via args):
    prob: The probability of a pixel being changed to Salt, Pepper, or staying unchanged.

    How it works:
    For each pixel in the image, generate a random number.
    If the random number is less than prob, change the pixel value to Salt (255).
    If the random number is greater than 1 - prob, change the pixel value to Pepper (0).
    Otherwise, leave the pixel unchanged.

  '''
  ###############################################
  ############# YOUR CODE GOES HERE #############

  if noise_type == 0: # Gaussian Noise
    ...

  else: # Salt-and-Pepper Noise
    ...

  ###############################################
  return output


In [ ]:
###############################################
######## PLOT INPUT AND OUTPUT IMAGES##########


###############################################

In [ ]:
def denoise(noisy_image):

  '''
  input:
    noisy_image (ndarray): the noisy image

  output:
    output_image (ndarray): denoised image

  '''
  ###############################################
  ############# YOUR CODE GOES HERE #############

  output = ...

  ###############################################
  return output



In [ ]:
###############################################
######## PLOT INPUT AND OUTPUT IMAGES##########


###############################################